<a href="https://colab.research.google.com/github/VSeema/Angular1/blob/master/Marks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install db-sqlite3

In [2]:
import pandas as pd
import sqlite3

In [3]:
db=sqlite3.connect("test2")
db.execute("drop table if exists Marks")
try:
  db.execute("create table Marks(Roll_No Integer,Name text,Marks Integer)")
except:
  print("already table existed")

In [4]:
db=sqlite3.connect("test2")

In [18]:
cmd="insert into Marks(Roll_No,Name,Marks) Values('5','suma','50')";
db.execute(cmd);
db.commit();

In [19]:
db=sqlite3.connect("test2")
rs=db.execute("Select * from Marks")
for row in rs:
  print(row)

(1, 'seema', 70)
(2, 'siddesh', 80)
(3, 'shobha', 90)
(4, 'viswa', 100)
(5, 'suma', 50)


In [20]:
db=sqlite3.connect("test2")
rs=db.execute("Select * from Marks where Name='seema'")
for row in rs:
  print(row)

(1, 'seema', 70)


In [21]:
db=sqlite3.connect("test2")
rs=db.execute("Select count(*) from Marks")
for row in rs:
  print(row)

(5,)


In [24]:
db=sqlite3.connect("test2")
rs=db.execute("Select Avg(Marks) from Marks")
for row in rs:
  print(row)


(78.0,)


In [25]:
db=sqlite3.connect("test2")
rs=db.execute("Select SUM(Marks) from Marks")
for row in rs:
  print(row)

(390,)


In [3]:
db=sqlite3.connect("test2")
qry ="""
Select * from Marks
"""
mark=pd.read_sql_query(qry,db)
mark

,Roll_No,Name,Marks
0,1,seema,70
1,2,siddesh,80
2,3,shobha,90
3,4,viswa,100
4,5,suma,50


In [27]:
! git clone https://github.com/google-research/tapas.git

Cloning into 'tapas'...
remote: Enumerating objects: 442, done.
remote: Total 442 (delta 0), reused 0 (delta 0), pack-reused 442
Receiving objects: 100% (442/442), 298.94 KiB | 1000.00 KiB/s, done.
Resolving deltas: 100% (276/276), done.


In [28]:
! pip install ./tapas

Processing ./tapas
     |████████████████████████████████| 3.5MB 12.2MB/s 
     |████████████████████████████████| 10.1MB 35.7MB/s 
     |████████████████████████████████| 516.2MB 32kB/s 
     |████████████████████████████████| 716kB 41.9MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 3.5MB 41.1MB/s 
     |████████████████████████████████| 358kB 49.8MB/s 
     |████████████████████████████████| 1.4MB 51.5MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 63.2MB 74kB/s 
     |████████████████████████████████| 225kB 51.1MB/s 
     |████████████████████████████████| 153kB 47.3MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 1.2MB 42.4MB/s 
     |████████████████████████████████| 184kB 52.5MB/s 
     |████████████████████████████████| 153kB 50.8MB/s 
     |███████████████████████████████

In [4]:
! gsutil cp gs://tapas_models/2020_04_21/tapas_sqa_base.zip . && unzip tapas_sqa_base.zip

Copying gs://tapas_models/2020_04_21/tapas_sqa_base.zip...
/ [1 files][  1.0 GiB/  1.0 GiB]   22.1 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      
Archive:  tapas_sqa_base.zip
   creating: tapas_sqa_base/
  inflating: tapas_sqa_base/model.ckpt.data-00000-of-00001  
  inflating: tapas_sqa_base/model.ckpt.index  
  inflating: tapas_sqa_base/README.txt  
  inflating: tapas_sqa_base/vocab.txt  
  inflating: tapas_sqa_base/bert_config.json  
  inflating: tapas_sqa_base/model.ckpt.meta  


In [5]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

In [6]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

In [7]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [8]:
mark = mark.astype(str)

In [9]:
mark.head()


,Roll_No,Name,Marks
0,1,seema,70
1,2,siddesh,80
2,3,shobha,90
3,4,viswa,100
4,5,suma,50


In [10]:
list_of_list = [[]]
list_of_list[0] = list(mark.columns)
list_of_list.extend(mark.values.tolist())

In [11]:
list_of_list

[['Roll_No', 'Name', 'Marks'],
 ['1', 'seema', '70'],
 ['2', 'siddesh', '80'],
 ['3', 'shobha', '90'],
 ['4', 'viswa', '100'],
 ['5', 'suma', '50']]

In [12]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = table_data
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python tapas/tapas/run_task_main.py \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

In [16]:
result = predict(list_of_list, ["what were the names?",
      "what is the total marks?",
      "what is the highest marks?",
      "how many marks has siddesh scored?",
      "what is the sum of the all marks?","what is the Roll No of viswa?","what is the Average marks of all the students?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Roll_No,Name,Marks
1,seema,70
2,siddesh,80
3,shobha,90
4,viswa,100
5,suma,50



> what were the names?
seema, viswa, shobha, suma, siddesh
> what is the total marks?
80, 100, 70, 90
> what is the highest marks?
100
> how many marks has siddesh scored?
80
> what is the sum of the all marks?
50
> what is the Roll No of viswa?
4
> what is the Average marks of all the students?
100
